### Tecnolgías del Lenguaje. Entregable 4
---
#### Entrenamiento de modelos regresores con embeddings

En este notebook se entrenan y evalúan modelos de regresión para predecir los cinco rasgos de personalidad (Big Five) a partir de embeddings de posts de usuarios generados por modelos de lenguaje.

La limpieza de datos es fundamental en este problema y no hay una solución óptima, por lo que utilizaremos los dos datasets de tipo "top50" generados con MPNet que recogen los 50 posts más relevantes para cada rasgo y de cada usuario, sin tener en cuenta la polaridad. Uno de los datasets tiene un umbral de 0.3, por lo que es menos restrictivo y tiene 140 000 filas, y el otro tiene un umbral de 0.4, por lo que es más restrictivo y tiene 40 000 filas. **No utilizaremos polaridad, aunque fue probado en una fase previa para los mismos regresores, pero obtuvimos resultados insatisfactorios.** Se utilizan tres tipos de regresores: **XGBoost**, **Random Forest** y **Ridge**, adaptando los hiperparámetros según el tamaño del dataset.

En primer lugar, a partir de los conjuntos de datos mecionados, agrupamos los posts por usuario y calculamos un embedding por cada usuario que codifique su *body* de posts.

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

model = SentenceTransformer("all-mpnet-base-v2", device="cuda")
model.half() # reduce VRAM

# Cargar datos
df = pd.read_csv("top50_posts_per_user_MPNet_by_trait_reduced.csv")
df_mt = pd.read_csv("top50_posts_per_user_MPNet_by_trait_reduced_more_threshold.csv")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Calculamos los embeddings de los posts de cada usuario usando el mismo modelo de lenguaje (MPNet) que habíamos utilizado para reducir el dataset, normalizando los vectores para mejorar la comparabilidad. Primero, debemos generar embeddings post por post, y luego se agregan por usuario mediante un promedio de todos los embeddings de sus posts, obteniendo un vector único por usuario que representa su contenido textual.

In [2]:
import torch
print(torch.cuda.is_available())

# Calcular embedding por post
embeddings = model.encode(
    df["post"].tolist(),
    batch_size=128,
    show_progress_bar=True,
    normalize_embeddings=True     # recomendado para MPNet
)
df["embedding"] = list(embeddings)

embeddings_mt = model.encode(df_mt["post"].tolist(), batch_size=128, show_progress_bar=True, normalize_embeddings=True)
df_mt["embedding"] = list(embeddings_mt)

# Calcular los embeddings por usuario
user_embeddings = (
    df.groupby("username")["embedding"]
      .apply(lambda x: np.mean(np.vstack(x), axis=0))
      .reset_index()
)

user_embeddings_mt = (
    df_mt.groupby("username")["embedding"]
      .apply(lambda x: np.mean(np.vstack(x), axis=0))
      .reset_index()
)

# Vector dimensionality
embed_dim = len(user_embeddings["embedding"].iloc[0])
print(f"Dimensión del embedding por usuario: {embed_dim}")

True


Batches:   0%|          | 0/1097 [00:00<?, ?it/s]

Batches:   0%|          | 0/301 [00:00<?, ?it/s]

Dimensión del embedding por usuario: 768


 La dimensión de los embeddings de usuario es 768, y servirá como input fijo para los modelos de regresión posteriores. Guardaremos los embeddings como .pkl para no tener que recalcularlos.

In [3]:
import pickle

# Guardar
df.to_pickle("embeddings.pkl")
df_mt.to_pickle("embeddings_mt.pkl")
print("Embeddings guardados en embeddings.pkl y embeddings_mt.pkl")

df.head()

Embeddings guardados en embeddings.pkl y embeddings_mt.pkl


,username,trait,post,similarity,embedding
0,-Areopagan-,Agreeableness,"I have two friends. I alienate everyone, event...",0.502757,"[0.04245, 0.02724, 0.006516, 0.036, -0.004215,..."
1,-Areopagan-,Agreeableness,I am smarter than you and will work you into d...,0.437536,"[0.0235, 0.02602, -0.00786, -0.00944, 0.02873,..."
2,-Areopagan-,Agreeableness,Yeah I wouldnt want to deal with someone like ...,0.388882,"[0.05643, 0.0437, 0.02481, -0.009125, -0.00469..."
3,-Areopagan-,Openness,"I have two friends. I alienate everyone, event...",0.300504,"[0.04245, 0.02724, 0.006516, 0.036, -0.004215,..."
4,-Areopagan-,Conscientiousness,I am smarter than you and will work you into d...,0.379553,"[0.0235, 0.02602, -0.00786, -0.00944, 0.02873,..."


A continuación, se cargan los embeddings previamente calculados desde archivos pickle y se convierten de listas a arrays de NumPy para facilitar su manipulación. Luego, los embeddings de usuario se expanden en columnas individuales, creando matrices donde cada columna representa una dimensión del embedding. Estas matrices deben ser transformadas en DataFrames de Pandas, con los nombres de columna indicativos (`emb_0`, `emb_1`, …) y se añaden los usernames correspondientes, dejando los DataFrames listos para usarse como features de entrada en los modelos de regresión.

In [5]:
df = pd.read_pickle("embeddings.pkl")
df_mt = pd.read_pickle("embeddings_mt.pkl")

# Convertir listas a arrays numpy
df["embedding"] = df["embedding"].apply(lambda x: np.array(x))
df_mt["embedding"] = df_mt["embedding"].apply(lambda x: np.array(x))

# Expandir el vector en columnas
emb_matrix = np.vstack(user_embeddings["embedding"].values)
emb_matrix_mt = np.vstack(user_embeddings_mt["embedding"].values)

# Convertir los embeddings a un DataFrame de Pandas
emb_df = pd.DataFrame(
    emb_matrix,
    columns=[f"emb_{i}" for i in range(embed_dim)]
)
emb_df_mt = pd.DataFrame(emb_matrix_mt, columns=[f"emb_{i}" for i in range(embed_dim)])

# Añadir usernames
emb_df["username"] = user_embeddings["username"].values
emb_df_mt["username"] = user_embeddings_mt["username"].values

Cargamos el *ground_truth*. Tiene pocos *missing_values*, por lo que imputaremos la media.

In [6]:
truth = pd.read_csv("authors_train.csv")

traits = [
    "agreeableness",
    "openness",
    "conscientiousness",
    "extraversion",
    "neuroticism"
]

# Imputar la media por columna en los datos faltantes
for col in traits:
    mean_value = truth[col].mean()
    truth[col].fillna(mean_value, inplace=True)

/tmp/ipython-input-3295801460.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  truth[col].fillna(mean_value, inplace=True)


Ya podemos entrenar los modelos regresores para predecir los valores de los rasgos para cada autor. Utilizaremos las siguientes celdas para calcular un modelo por rasgo, para cada archivo (umbral 0.3 y umbral 0.4). Esto se repetirá para los modelos RandomForest, XGBoost y Ridge.

#### **RandomForest (threshold = 0.3)**

Para Random Forest se ajustan parámetros como profundidad máxima y número mínimo de muestras por hoja para mejorar estabilidad.

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

models = {}

for t in traits:
    print(f"\n=== Entrenando modelo para {t} ===")

    # Fusionar embeddings con truth
    data = emb_df.merge(truth[["username", t]], on="username")

    X = data.drop(columns=["username", t])
    y = data[t]

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Random Forest optimizado
    model = RandomForestRegressor(
        n_estimators=700,           # más árboles para mayor precisión
        max_depth=18,               # limita profundidad para reducir overfitting
        min_samples_leaf=3,         # evita splits con muy pocos datos
        max_features=0.4,           # 40% de features por split, útil con embeddings densos
        n_jobs=-1,
        random_state=42,
        bootstrap=True
    )

    # Entrenar
    model.fit(X_train, y_train)

    # Predicciones
    preds = model.predict(X_test)

    # Métricas
    mse = mean_squared_error(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    pearson_corr, _ = pearsonr(y_test, preds)

    print(f"MSE: {mse:.2f}  |  MAE: {mae:.2f}  |  R²: {r2:.4f}  |  Pearson: {pearson_corr:.4f}")

    models[t] = model


# Guardar modelos
for trait, model in models.items():
    with open(f"rf_embeddings_{trait}.pkl", "wb") as f:
        pickle.dump(model, f)

print("\nModelos guardados como rf_embeddings_[trait].pkl")



=== Entrenando modelo para agreeableness ===
MSE: 881.66  |  MAE: 25.91  |  R²: 0.0314  |  Pearson: 0.2067

=== Entrenando modelo para openness ===
MSE: 793.32  |  MAE: 23.90  |  R²: 0.0395  |  Pearson: 0.2014

=== Entrenando modelo para conscientiousness ===
MSE: 856.58  |  MAE: 25.53  |  R²: -0.0043  |  Pearson: 0.1196

=== Entrenando modelo para extraversion ===
MSE: 1032.45  |  MAE: 28.38  |  R²: 0.0599  |  Pearson: 0.2479

=== Entrenando modelo para neuroticism ===
MSE: 914.60  |  MAE: 26.57  |  R²: 0.0961  |  Pearson: 0.3422

Modelos guardados como rf_embeddings_[trait].pkl


#### **RandomForest (threhold = 0.4)**

In [18]:
models_mt = {}

for t in traits:
    print(f"\n=== Entrenando modelo para {t} ===")

    # Fusionar embeddings con truth
    data_mt = emb_df_mt.merge(truth[["username", t]], on="username")

    X = data_mt.drop(columns=["username", t])
    y = data_mt[t]

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Random Forest optimizado para 40k filas
    model = RandomForestRegressor(
        n_estimators=500,       # suficiente para estabilidad
        max_depth=15,           # limita profundidad para evitar overfitting
        min_samples_leaf=4,     # evita splits con muy pocos datos
        max_features=0.3,       # 30% de features por split
        n_jobs=-1,
        random_state=42,
        bootstrap=True
    )

    # Entrenar
    model.fit(X_train, y_train)

    # Predicciones
    preds = model.predict(X_test)

    # Métricas
    mse = mean_squared_error(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    pearson_corr, _ = pearsonr(y_test, preds)

    print(f"MSE: {mse:.2f}  |  MAE: {mae:.2f}  |  R²: {r2:.4f}  |  Pearson: {pearson_corr:.4f}")

    models_mt[t] = model


# Guardar modelos
for trait, model in models_mt.items():
    with open(f"rf_embeddings_mt_{trait}.pkl", "wb") as f:
        pickle.dump(model, f)

print("\nModelos guardados como rf_embeddings_mt_[trait].pkl")



=== Entrenando modelo para agreeableness ===
MSE: 853.24  |  MAE: 25.48  |  R²: 0.0628  |  Pearson: 0.2957

=== Entrenando modelo para openness ===
MSE: 718.48  |  MAE: 22.32  |  R²: 0.0101  |  Pearson: 0.1520

=== Entrenando modelo para conscientiousness ===
MSE: 815.74  |  MAE: 24.13  |  R²: 0.0679  |  Pearson: 0.2703

=== Entrenando modelo para extraversion ===
MSE: 899.90  |  MAE: 26.45  |  R²: 0.0258  |  Pearson: 0.2120

=== Entrenando modelo para neuroticism ===
MSE: 1062.17  |  MAE: 28.95  |  R²: 0.0224  |  Pearson: 0.1857

Modelos guardados como rf_embeddings_mt_[trait].pkl


#### **XGBoost (threshold = 0.3)**

Para XGBoost se emplea early stopping sobre un conjunto de validación para optimizar la cantidad de árboles y evitar sobreajuste

In [12]:
import xgboost as xgb

models = {}

for t in traits:
    print(f"\n=== Entrenando modelo para {t} ===")

    # Fusionar embeddings con el truth
    data = emb_df_mt.merge(truth[["username", t]], on="username")

    X = data.drop(columns=["username", t])
    y = data[t]

    # Split Train + Test
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Split Train_full → Train + Val
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full,
        test_size=0.1,  # 10% del train_full → ~8% total
        random_state=42
    )

    # Convertir a DMatrix (XGBoost nativo)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Parámetros
    params = {
        "objective": "reg:squarederror",
        "learning_rate": 0.015,
        "max_depth": 4,
        "min_child_weight": 8,
        "subsample": 0.8,
        "colsample_bytree": 0.25,
        "colsample_bylevel": 0.25,
        "reg_alpha": 1.0,
        "reg_lambda": 2.0,
        "eval_metric": "rmse",
        "seed": 42
    }

    # Entrenar con early stopping usando la validación
    evallist = [(dtrain, "train"), (dval, "eval")]

    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=1200,
        evals=evallist,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predicciones para test
    preds = bst.predict(dtest)

    # Métricas
    mse = mean_squared_error(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    pearson_corr, _ = pearsonr(y_test, preds)

    print(f"MSE: {mse:.2f}  |  MAE: {mae:.2f}  |  R²: {r2:.4f}  |  Pearson: {pearson_corr:.4f}")

    models[t] = bst

# Guardar modelos
for trait, model in models.items():
    with open(f"xgb_embeddings_{trait}.pkl", "wb") as f:
        pickle.dump(model, f)

print("\nModelos guardados como xgb_embeddings_[trait].pkl")



=== Entrenando modelo para agreeableness ===
MSE: 860.14  |  MAE: 25.69  |  R²: 0.0552  |  Pearson: 0.2731

=== Entrenando modelo para openness ===
MSE: 710.26  |  MAE: 22.03  |  R²: 0.0214  |  Pearson: 0.1548

=== Entrenando modelo para conscientiousness ===
MSE: 809.69  |  MAE: 24.30  |  R²: 0.0748  |  Pearson: 0.2866

=== Entrenando modelo para extraversion ===
MSE: 895.46  |  MAE: 26.27  |  R²: 0.0306  |  Pearson: 0.2008

=== Entrenando modelo para neuroticism ===
MSE: 1096.31  |  MAE: 29.13  |  R²: -0.0090  |  Pearson: 0.1333

Modelos guardados como xgb_embeddings_[trait].pkl


#### **XGBoost (threshold = 0.4)**

In [13]:
models_mt = {}

for t in traits:
    print(f"\n=== Entrenando modelo para {t} ===")

    # Fusionar embeddings con el truth
    data_mt = emb_df_mt.merge(truth[["username", t]], on="username")

    X = data_mt.drop(columns=["username", t])
    y = data_mt[t]

    # Split Train + Test
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Split Train_full → Train + Val
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full,
        test_size=0.1,  # 10% del train_full → ~8% total
        random_state=42
    )

    # Convertir a DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Parámetros
    params = {
        "objective": "reg:squarederror",
        "learning_rate": 0.015,
        "max_depth": 3,
        "min_child_weight": 12,
        "subsample": 0.8,
        "colsample_bytree": 0.2,
        "colsample_bylevel": 0.25,
        "reg_alpha": 2.0,
        "reg_lambda": 3.0,
        "eval_metric": "rmse",
        "seed": 42
    }

    evallist = [(dtrain, "train"), (dval, "eval")]

    # Entrenar con early stopping usando la validación
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=800,
        evals=evallist,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predicciones para test
    preds = bst.predict(dtest)

    # Métricas
    mse = mean_squared_error(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    pearson_corr, _ = pearsonr(y_test, preds)

    print(f"MSE: {mse:.2f}  |  MAE: {mae:.2f}  |  R²: {r2:.4f}  |  Pearson: {pearson_corr:.4f}")

    models_mt[t] = bst

# Guardar modelos
for trait, model in models.items():
    with open(f"xgb_embeddings_mt_{trait}.pkl", "wb") as f:
        pickle.dump(model, f)

print("\nModelos guardados como xgb_embeddings_mt_[trait].pkl")



=== Entrenando modelo para agreeableness ===
MSE: 877.51  |  MAE: 26.01  |  R²: 0.0361  |  Pearson: 0.2224

=== Entrenando modelo para openness ===
MSE: 705.47  |  MAE: 21.99  |  R²: 0.0280  |  Pearson: 0.1685

=== Entrenando modelo para conscientiousness ===
MSE: 820.26  |  MAE: 24.37  |  R²: 0.0627  |  Pearson: 0.2560

=== Entrenando modelo para extraversion ===
MSE: 880.47  |  MAE: 26.08  |  R²: 0.0468  |  Pearson: 0.2397

=== Entrenando modelo para neuroticism ===
MSE: 1089.06  |  MAE: 29.03  |  R²: -0.0023  |  Pearson: 0.1324

Modelos guardados como xgb_embeddings_mt_[trait].pkl


#### Ridge (threshold = 0.3)

Para Ridge se realiza una búsqueda en un grid de valores de alpha para minimizar el RMSE.

In [20]:
from sklearn.linear_model import Ridge

# Grid de alpha para probar
alpha_grid = np.logspace(-3, 3, 100)  # 0.001 a 1000, 100 valores

models = {}

for t in traits:
    print(f"\n=== Selección de mejor Ridge para {t} ===")

    # Fusionar embeddings y truth
    data = emb_df.merge(truth[["username", t]], on="username")

    X = data.drop(columns=["username", t])
    y = data[t]

    # Dividir dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    best_mse = np.inf
    best_model = None
    best_metrics = {}

    for alpha in alpha_grid:
        model = Ridge(alpha=alpha, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        # Calcular métricas
        mse = mean_squared_error(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        pearson_corr, _ = pearsonr(y_test, preds)

        # Selección del mejor modelo por RMSE
        if mse < best_mse:
            best_rmse = mse
            best_model = model
            best_metrics = {"MSE": mse, "MAE": mae, "R2": r2, "Pearson": pearson_corr}

    print(f"MSE: {best_metrics['MSE']:.2f}  |  MAE: {best_metrics['MAE']:.2f}  |  R²: {best_metrics['R2']:.4f}  |  Pearson: {best_metrics['Pearson']:.4f}")
    models[t] = best_model

# Guardar los mejores modelos por rasgo
for trait, model in models.items():
    with open(f"best_ridge_{trait}.pkl", "wb") as f:
        pickle.dump(model, f)

print("\nMejores modelos Ridge guardados como best_ridge_[trait].pkl")



=== Selección de mejor Ridge para agreeableness ===
MSE: 915.44  |  MAE: 26.57  |  R²: -0.0057  |  Pearson: 0.1135

=== Selección de mejor Ridge para openness ===
MSE: 826.98  |  MAE: 24.16  |  R²: -0.0013  |  Pearson: 0.1173

=== Selección de mejor Ridge para conscientiousness ===
MSE: 856.89  |  MAE: 25.66  |  R²: -0.0046  |  Pearson: 0.1825

=== Selección de mejor Ridge para extraversion ===
MSE: 1099.42  |  MAE: 29.33  |  R²: -0.0011  |  Pearson: 0.0683

=== Selección de mejor Ridge para neuroticism ===
MSE: 1012.47  |  MAE: 28.12  |  R²: -0.0006  |  Pearson: 0.1664

Mejores modelos Ridge guardados como best_ridge_[trait].pkl


#### Ridge (threshold = 0.4)

In [21]:
from sklearn.linear_model import Ridge

models_mt = {}

for t in traits:
    print(f"\n=== Selección de mejor Ridge para {t} ===")

    # Fusionar embeddings y truth
    data_mt = emb_df_mt.merge(truth[["username", t]], on="username")

    X = data_mt.drop(columns=["username", t])
    y = data_mt[t]

    # Dividir dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    best_mse = np.inf
    best_model = None
    best_metrics = {}

    for alpha in alpha_grid:
        model = Ridge(alpha=alpha, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        # Calcular métricas
        mse = mean_squared_error(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        pearson_corr, _ = pearsonr(y_test, preds)

        # Selección del mejor modelo por RMSE
        if mse < best_mse:
            best_mse = mse
            best_model = model
            best_metrics = {"MSE": mse, "MAE": mae, "R2": r2, "Pearson": pearson_corr}

    print(f"MSE: {best_metrics['MSE']:.2f}  |  MAE: {best_metrics['MAE']:.2f}  |  R²: {best_metrics['R2']:.4f}  |  Pearson: {best_metrics['Pearson']:.4f}")
    models_mt[t] = best_model

# Guardar los mejores modelos por rasgo
for trait, model in models_mt.items():
    with open(f"best_ridge_mt_{trait}.pkl", "wb") as f:
        pickle.dump(model, f)

print("\nMejores modelos Ridge guardados como best_ridge_mt_[trait].pkl")



=== Selección de mejor Ridge para agreeableness ===
MSE: 870.68  |  MAE: 25.72  |  R²: 0.0436  |  Pearson: 0.2371

=== Selección de mejor Ridge para openness ===
MSE: 715.44  |  MAE: 22.18  |  R²: 0.0143  |  Pearson: 0.1245

=== Selección de mejor Ridge para conscientiousness ===
MSE: 828.42  |  MAE: 24.40  |  R²: 0.0534  |  Pearson: 0.2334

=== Selección de mejor Ridge para extraversion ===
MSE: 916.27  |  MAE: 26.73  |  R²: 0.0081  |  Pearson: 0.1375

=== Selección de mejor Ridge para neuroticism ===
MSE: 1084.90  |  MAE: 29.04  |  R²: 0.0015  |  Pearson: 0.1075

Mejores modelos Ridge guardados como best_ridge_mt_[trait].pkl


#### **Baseline utilizando la media y desviación.**

La predicción se basa únicamente en la media del rasgo en el conjunto de entrenamiento, a la que se añade una pequeña variación aleatoria proporcional a la desviación estándar, simulando ligeras oscilaciones sin usar ninguna información de los embeddings. Calculamos las mismas métricas de desempeño (MSE, MAE, R² y correlación de Pearson) sobre el conjunto de test para tener un punto de referencia.

In [16]:
models_baseline = {}

for t in traits:
    print(f"\n=== Modelo baseline conservador para {t} ===")

    # Fusionar embeddings y truth
    data = emb_df.merge(truth[["username", t]], on="username")

    X = data.drop(columns=["username", t])
    y = data[t]

    # Dividir dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Media y desviación estándar del rasgo en train
    mean_y = y_train.mean()
    std_y = y_train.std()

    # Predicción: media + pequeña variación aleatoria
    np.random.seed(42)
    small_factor = 0.05  # porcentaje de oscilación respecto a std
    preds = mean_y + np.random.normal(0, std_y * small_factor, size=len(y_test))

    # Métricas
    mse = mean_squared_error(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    pearson_corr, p_value = pearsonr(y_test, preds)

    print(f"MSE: {mse:.2f}  |  MAE: {mae:.2f}  |  R²: {r2:.4f}  |  Pearson: {pearson_corr:.4f}")

    # Guardar modelo (solo media y factor de variación)
    models_baseline[t] = {"mean": mean_y, "small_factor": small_factor}

# Guardar modelos baseline
with open("baseline_mean_variation_models.pkl", "wb") as f:
    pickle.dump(models_baseline, f)

print("\nModelos baseline guardados como baseline_mean_variation_models.pkl")



=== Modelo baseline conservador para agreeableness ===
MSE: 921.45  |  MAE: 26.67  |  R²: -0.0123  |  Pearson: -0.0377

=== Modelo baseline conservador para openness ===
MSE: 830.19  |  MAE: 24.21  |  R²: -0.0052  |  Pearson: -0.0137

=== Modelo baseline conservador para conscientiousness ===
MSE: 858.63  |  MAE: 25.68  |  R²: -0.0067  |  Pearson: 0.0084

=== Modelo baseline conservador para extraversion ===
MSE: 1096.99  |  MAE: 29.23  |  R²: 0.0011  |  Pearson: 0.0522

=== Modelo baseline conservador para neuroticism ===
MSE: 1020.97  |  MAE: 28.22  |  R²: -0.0090  |  Pearson: -0.0563

Modelos baseline guardados como baseline_mean_variation_models.pkl


### **Ensemble de modelos**

En base a los resultados anteriores, guardamos los mejores modelos por rasgo para calcular un ensemble.

In [23]:
model_paths = {
    "agreeableness": "rf_embeddings_mt_agreeableness.pkl",
    "openness": "xgb_embeddings_mt_openness.pkl",
    "conscientiousness": "xgb_embeddings_conscientiousness.pkl",
    "extraversion": "xgb_embeddings_mt_extraversion.pkl",
    "neuroticism": "rf_embeddings_neuroticism.pkl",
}

models_ensemble = {}

for trait, path in model_paths.items():
    with open(path, "rb") as f:
        models_ensemble[trait] = pickle.load(f)

print("Modelos cargados para el ensemble.")

Modelos cargados para el ensemble.


# **Predicción**

In [32]:
import pandas as pd
import pickle
import numpy as np
import xgboost as xgb

# -----------------------------
# Archivos de entrada (por rasgo)
# -----------------------------
embedding_files = {
    "agreeableness": "embeddings_mt.pkl",
    "openness": "embeddings_mt.pkl",
    "conscientiousness": "embeddings.pkl",
    "extraversion": "embeddings_mt.pkl",
    "neuroticism": "embeddings.pkl"
}

# -----------------------------
# Modelos del ensemble
# -----------------------------
model_paths = {
    "agreeableness": "rf_embeddings_mt_agreeableness.pkl",
    "openness": "xgb_embeddings_mt_openness.pkl",
    "conscientiousness": "xgb_embeddings_conscientiousness.pkl",
    "extraversion": "xgb_embeddings_mt_extraversion.pkl",
    "neuroticism": "rf_embeddings_neuroticism.pkl"
}

traits = list(model_paths.keys())

authors_test_file = "authors_test.csv"
output_file = "predicciones_ensemble_BSP.csv"

# -----------------------------
# Cargar lista completa de autores de test
# -----------------------------
authors_test = pd.read_csv(authors_test_file)  # columna 'username'

# -----------------------------
# Preparar dataframe final
# -----------------------------
predictions = authors_test[["username"]].copy()

# =============================
#   PREDICCIÓN POR CADA RASGO
# =============================

for trait in traits:
    print(f"\n=== Procesando {trait.upper()} ===")

    # --------------------------------------
    # 1. Cargar embeddings correctos para el rasgo
    # --------------------------------------
    emb_file = embedding_files[trait]
    print(f"Usando embeddings: {emb_file}")
    df = pd.read_pickle(emb_file)

    # --------------------------------------
    # 2. Agregar embeddings por usuario
    # --------------------------------------
    user_embeddings = (
        df.groupby("username")["embedding"]
          .apply(lambda x: np.mean(np.vstack(x), axis=0))
          .reset_index()
    )

    # --------------------------------------
    # 3. Merge con autores de test (mantener orden + usuarios sin embedding)
    # --------------------------------------
    user_embeddings = authors_test[["username"]].merge(
        user_embeddings, on="username", how="left"
    )

    # --------------------------------------
    # 4. Rellenar embeddings faltantes
    # --------------------------------------
    embedding_mean = np.mean(
        np.vstack(user_embeddings["embedding"].dropna().values), axis=0
    )

    user_embeddings["embedding"] = user_embeddings["embedding"].apply(
        lambda x: x if isinstance(x, np.ndarray) else embedding_mean
    )

    # --------------------------------------
    # 5. Convertir a DataFrame con nombres de columnas
    # --------------------------------------
    embed_dim = len(user_embeddings["embedding"].iloc[0])
    X_test = pd.DataFrame(
        np.vstack(user_embeddings["embedding"].values),
        columns=[f"emb_{i}" for i in range(embed_dim)]
    )

    feature_names = list(X_test.columns)

    # --------------------------------------
    # 6. Cargar modelo correcto para el rasgo
    # --------------------------------------
    with open(model_paths[trait], "rb") as f:
        model = pickle.load(f)

    # --------------------------------------
    # 7. Predicción
    # --------------------------------------
    if "xgboost" in str(type(model)).lower():
        # XGBoost → necesita DMatrix
        dtest = xgb.DMatrix(X_test, feature_names=feature_names)
        y_pred = model.predict(dtest)
    else:
        # RandomForest → usa DataFrame directamente
        y_pred = model.predict(X_test)

    predictions[trait] = y_pred


# -----------------------------
# Añadir columnas fijas y reordenar
# -----------------------------
predictions["team_name"] = "BSP"
predictions["variant_name"] = "ensemble_best_per_trait"

predictions = predictions[
    ["team_name", "variant_name", "username"] + traits
]

# -----------------------------
# Guardar CSV final
# -----------------------------
predictions.to_csv(output_file, index=False)
print(f"\nArchivo de predicciones generado: {output_file} ({len(predictions)} autores)")



=== Procesando AGREEABLENESS ===
Usando embeddings: embeddings_mt.pkl

=== Procesando OPENNESS ===
Usando embeddings: embeddings_mt.pkl

=== Procesando CONSCIENTIOUSNESS ===
Usando embeddings: embeddings.pkl

=== Procesando EXTRAVERSION ===
Usando embeddings: embeddings_mt.pkl

=== Procesando NEUROTICISM ===
Usando embeddings: embeddings.pkl

Archivo de predicciones generado: predicciones_ensemble_BSP.csv (323 autores)
